# AllServicesMonitor
- 所有服务监控器-AAA防护系统的第三个部分
- 负责监控所有服务的状态，出现异常根据现象自动处理并报告

In [ ]:
# main

import os
import json
import time


class Base:
    
    def __init__(self):
        
        self.log = Log()
    
    def execute_command(self, command):

        terminal = os.popen(command)
        a = terminal.read()
        terminal.close()
        return a

    def send_email(self, title, content, address):

        self.execute_command("echo %s | mail -s '%s' %s" % (content, title, address))


# desc: template_1/2 是负责系统应用层面服务的检查，有httpd/nginx/docker/emby/jupyterlab/docker-container/python等
class SystemLevelServices:
    
    def __init__(self, base):
        
        self.base = base
        self.log = self.base.log
        self.execute_command = self.base.execute_command
        self.send_email = self.base.send_email
    
    def template_1(self, name, ps, condition, restart):

        # template_1
        status = True
        # check the process
        ps_result = self.execute_command(ps)
        count = 0
        while condition not in ps_result and count <= 3:
            print("WARNING: %s IS DOWN, try to restart" % name)
            time.sleep(3)
            status = False
            count+=1
            self.execute_command(restart)
            ps_result = self.execute_command(ps)

        if count <= 3:
            status = True

        if not httpd_status:
            print("ERROR: CAN'T ENABLE THE %s, please check it" % name)
            self.send_email("ERROR: %s down but can't restart" % name, "From HadreamServer: \n ERROR: %s down but can't restart" % name, "1481605673@qq.com")
        else:
            self.log.add_log("fix the problem success, process-%s" % name, 1)

        return status

    def template_2(self, name, ps ,condition, restart):

        # template_2
        status = True
        # check the process
        ps_result = self.execute_command(ps)
        count = 0
        while condition not in ps_result and count <= 3:
            print("WARNING: %s IS DOWN, is docker down?" % name)
            # is docker down?
            if self.template_1("DOCKER", "systemctl status docker", "0", "systemctl restart docker"):
                time.sleep(3)
                status = False
                count+=1
                result = self.execute_command(restart)
                ps_result = self.execute_command(ps)
            else:
                status = False
                count = 4
                break

        if count <= 3:
            status = True

        if not status:
            print("ERROR: CAN'T ENABLE THE %s, please check it" % name)
            self.send_email("ERROR: %s down but can't restart" % name, "From HadreamServer: \n ERROR: %s down but can't restart" % name, "1481605673@qq.com")
        else:
            self.log.add_log("fix the problem success, process-%s" % name, 1)


In [37]:
# desc: template_3则是从网络上进行连接的测试，相对系统层面的处理来说，这个更为复杂
import requests


class NetworkLevelServices:
    
    def __init__(self, base):
        
        self.base = base
        self.log = self.base.log
        self.execute_command = self.base.execute_command
        self.send_email = self.base.send_email
        
        self.user_agents = [
            {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"},
            {"User-Agent": "Mozilla/5.0 (Windows NT 5.1; U; en; rv:1.8.1) Gecko/20061208 Firefox/2.0.0 Opera 9.50"},
            {"User-Agent": "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; en) Opera 9.50"},
            {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0"},
            {"User-Agent": "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10"},
            {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko"},
            {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2 "},
        ]
        
    def ping(self, address, mode="get", body=None, verify=False):
        
        
        headers = self.user_agents[random.randint(0, len(self.user_agents))]
        try:
            if mode == "post":
                r = requests.post(address, body, headers=headers, verify=verify)
            else:
                r = requests.get(address, headers=headers, verify=verify)
        except:
            code = -1
        else:
            code = r.status_code
            
        return code
    
    def httpd_503(self):
        
        """
        遇到http返回码503的拯救方案
        """
        self.execute_command("systemctl restart php-fpm-74")
        self.execute_command("systemctl restart mysqld")
        self.execute_command("systemctl restart httpd")
        
    def template_3(self, name, inner_address, external_ad_address, external_verify=True):
        
        # template_3
        inner_status = self.ping(inner_address)
        external_ad_status = self.ping(external_ad_address, verify=external_verify)
#         external_tp_status = self.ping("http://tencent.hadream.ltd:82/nextcloud")
        
        # inner
        count = 0
        while inner_status != 200 and count <=3:
            self.log.add_log("%s is down in code-%s!!! can't ping in the inner network, try to handle" % (name, inner_status), 3)
            if inner_status == 503:
                self.httpd_503()
                time.sleep(3)
            else:
                self.execute_command("systemctl restart httpd")
                time.sleep(5)
            
            inner_status = self.ping(inner_address)
            count+=1
        
        if inner_status != 200:
            self.log.add_log("ERROR: %s is in code-%s and can' be revived" % (name, inner_status), 3)
            self.send_email("ERROR: %s is in code-%s" % (name, inner_status), "ERROR: %s can't be revived automatically, need to check! \n code-%s, which happend in the innner network testing" % (name, inner_status),
                           "1481605673@qq.com")
        
        # externel_ali_domain_status
        count = 0
        while external_ad_status != 200 and count <=4:
            self.log.add_log("can't ping %s in the external network from ali_thread domain mode, code-%s!!! try to handle" % (name, external_ad_status), 3)
            if external_ad_status == -1:
                # nps down or service down
                if inner_status != 200:
                    # service down
                    self.log.add_log("ERROR: %s IS DOWN! exit, because the inner checking had been executed" % name, 3)
                    break
                else:
                    count+=1
                    # nps down
                    if count <= 2:
                        # try to reload npc
                        self.log.add_log("ERROR: NPS IS DOWN, try to fix by restarting npc", 1)
                        npc_pids = self.execute_command("ps -ef | grep npc | awk '{ print $2; }'")
                        self.execute_command("kill -9 %s" % npc_pids[0])
                        self.execute_command("kill -9 %s" % npc_pids[1])
                        time.sleep(1)
                        self.execute_command("sh /home/hadream/npc/run_npc.sh")
                        time.sleep(2)
                        external_ad_status = self.ping(external_ad_address, verify=external_verify)
                    else:
                        # try to reload nps and npc
                        self.log.add_log("try to reload nps", 1)
                        self.ping("http://hadream.ltd:6003/api", body={"command": "restart_nps"}, mode="post")
                        time.sleep(4)
                        self.log.add_log("ERROR: NPS IS DOWN, try to fix by restart npc", 1)
                        npc_pids = self.execute_command("ps -ef | grep npc | awk '{ print $2; }'")
                        self.execute_command("kill -9 %s" % npc_pids[0])
                        self.execute_command("kill -9 %s" % npc_pids[1])
                        time.sleep(1)
                        self.execute_command("sh /home/hadream/npc/run_npc.sh")
                        time.sleep(2)
                        external_ad_status = self.ping(external_ad_address, verify=external_verify)
                        
            else:
                self.log.add_log("can't handle, unknown reason: code-%s" % external_ad_status, 3)
                break
                
        if external_ad_status != 200:
            self.log.add_log("can't fix the problem!", 3)
            self.send_email("ERROR: %s is in code-%s" % (name, inner_status), "ERROR: %s can't be revived automatically, need to check! \n code-%s, which happend in the innner network testing" % (name, inner_status),
                           "1481605673@qq.com")
        else:
            self.log.add_log("fix the problem success, process-%s" % name, 1)
            
    def restart_nps(self, address, verify):
        
        # try to reload npc
        self.log.add_log("ERROR: NPS IS DOWN, try to fix by restarting npc", 1)
        npc_pids = self.execute_command("ps -ef | grep npc | awk '{ print $2; }'")
        self.execute_command("kill -9 %s" % npc_pids[0])
        self.execute_command("kill -9 %s" % npc_pids[1])
        time.sleep(1)
        self.execute_command("sh /home/hadream/npc/run_npc.sh")
        time.sleep(2)
        external_ad_status = self.ping(external_ad_address, verify=external_verify)

        time.sleep(5)
        
        if ping() != 200:
            # try to reload nps and npc
            self.log.add_log("try to reload nps", 1)
            self.ping("http://hadream.ltd:6003/api", body={"command": "restart_nps"}, mode="post")
            time.sleep(4)
            self.log.add_log("ERROR: NPS IS DOWN, try to fix by restart npc", 1)
            npc_pids = self.execute_command("ps -ef | grep npc | awk '{ print $2; }'")
            self.execute_command("kill -9 %s" % npc_pids[0])
            self.execute_command("kill -9 %s" % npc_pids[1])
            time.sleep(1)
            self.execute_command("sh /home/hadream/npc/run_npc.sh")
            time.sleep(2)
            external_ad_status = self.ping(external_ad_address, verify=external_verify)
        time.sleep(5)
        
        return ping()
    
    def revive_nps(self):
    
        """
        保障nps的绝对运行
        两个以上的服务无法访问且内部可以访问，则是nps断开
        """
        ex_ping1 = self.ping("https://office.hadream.ltd", verify=True)
        ex_ping2 = self.ping("https://hadream.ltd:82", verify=True)
        in_ping1 = self.ping("http://192.168.3.156:9000")
        in_ping2 = self.ping("http://192.168.3.156")
        
        if ex_ping1 != 200 and in_ping == 200:
            # onlyoffice service normal, but nps maybe down
            self.log.add_log("NPS MAYBE DOWN, try to restart", 3)
            if self.restart_nps("https://office.hadream.ltd", True) != 200:
                self.log.add_log("can't restart nps!!!", 3)
                self.send_email("ERROR: NPS IS DOWN BUT CAN'T RESTART", "nps down and can't restart!", "1481605673@qq.com")
            else:
                self.log.add_log("restart nps success", 1)
        elif ex_ping != 200 and in_ping != 200:
            # onlyoffice service down
            self.execute_command("docker run gracious_fermi")
            time.sleep(2)
            in_ping1 = self.ping("http://192.168.3.156:9000")
            if in_ping1 == 200:
                ex_ping1 = self.ping("https://office.hadream.ltd", verify=True)
                if ex_ping1 == 200:
                    pass
                else:
                    # nps down
                    self.log.add_log("NPS IS DOWN, try to restart", 3)
                    if self.restart_nps("https://office.hadream.ltd", True) != 200:
                        self.log.add_log("can't restart nps!!!", 3)
                        self.send_email("ERROR: NPS IS DOWN BUT CAN'T RESTART", "nps down and can't restart!", "1481605673@qq.com")
                    else:
                        self.log.add_log("restart nps success", 1)
            else:
                # can't restart the onlyoffice, check ping2
                if in_ping2 == 200 and ex_ping2 == 200:
                    # nps and httpd is normal, but onlyoffice down
                    self.log.add_log("ONLYOFFICE IS DOWN and can't fix", 3)
                elif in_ping2 == 200 and ex_ping != 200:
                    # nps is down
                    self.log.add_log("NPS IS DOWN, try to restart", 3)
                    if self.restart_nps("https://hadream.ltd:82", True) != 200:
                        self.log.add_log("can't restart nps!!!", 3)
                        self.send_email("ERROR: NPS IS DOWN BUT CAN'T RESTART", "nps down and can't restart!", "1481605673@qq.com")
                    else:
                        self.log.add_log("restart nps success", 1)
                elif in_ping2 != 200:
                    # httpd is down
                    self.log.add_log("HTTPD IS DOWN, try to restart as code-503", 3)
                    self.httpd_503()
                    in_ping2 = self.ping("http://192.168.3.156")
                    if in_ping2 == 200:
                        ex_ping2 = self.ping("https://hadream.ltd:82", verify=True)
                        if ex_ping2 == 200:
                            # nps is normal
                            pass
                        else:
                            # nps is down
                            self.log.add_log("NPS IS DOWN, try to restart", 3)
                            if self.restart_nps("https://hadream.ltd:82", True) != 200:
                                self.log.add_log("can't restart nps!!!", 3)
                                self.send_email("ERROR: NPS IS DOWN BUT CAN'T RESTART", "nps down and can't restart!", "1481605673@qq.com")
                            else:
                                self.log.add_log("restart nps success", 1)



In [25]:
import requests
headers={
        'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'
    }
r = requests.get("http://hadream.ltd", verify=True, headers=headers)
r.status_code

200

In [36]:
!ps -ef | grep npc

root      452722       1  0 7月15 ?       00:00:17 ./npc -server=8.210.36.99:8024 -vkey=Yy20050801056 -type=tcp
root      452738       1  0 7月15 ?       00:00:05 ./npc -server=159.75.114.34:8024 -vkey=Y20050801056 -type=tcp
root      559410  158929  0 00:23 pts/0    00:00:00 /bin/bash -c ps -ef | grep npc
root      559412  559410  0 00:23 pts/0    00:00:00 grep npc
